In [154]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [195]:
df = pd.read_csv('titanic.csv')
df.head(6)

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0
5,6,"Moran, Mr. James",3,male,NaN,0,0,330877,8.4583,NaN,Q,0


In [196]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [197]:
df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1


In [198]:
X_train,X_test, y_train, y_test = train_test_split(df.drop(['Survived'],axis=1), df['Survived'],test_size=.2)

In [199]:
#imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [200]:
#One-hot-encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore',dtype=np.int32),[1,6])
],remainder='passthrough')

In [201]:
#Scalling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [202]:
#Feature Selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [203]:
#Train the Model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [210]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [211]:
from sklearn import set_config
set_config(display='diagram')

In [212]:
pipe.fit(X_train,y_train)

D:\jupyter\mynumpy\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002605ACD44C0>)),
                ('trf5', DecisionTreeClassifier())])

In [213]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [214]:
y_pred = pipe.predict(X_test)

In [215]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6312849162011173

# Exporting Pipeline

In [222]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [232]:
model = pickle.load(open('pipe.pkl','rb'))

In [243]:
input_ = np.array([2,'male',33,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [244]:
input_

array([[2, 'male', 33, 0, 0, 10.5, 'S']], dtype=object)

In [245]:
model.predict(input_)

D:\jupyter\mynumpy\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
D:\jupyter\mynumpy\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)